In [ ]:
def extract_caption_pairs(pdf_paths, caption_save_path):
    results = [] 

    for file_index, pdf_path in enumerate(pdf_paths):
        doc = fitz.open(pdf_path)
        img_index = 1

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            blocks = page.get_text("blocks")

            text_blocks = [
                b for b in blocks
                if b[6] == 0 and b[4].strip() != ''
            ]

            for i, block in enumerate(text_blocks):
                block_text = block[4]

                if re.search(r'Figure\s*\d+', block_text, re.IGNORECASE):
                    prev_block_text = None

                    j = i - 1
                    while j >= 0:
                        candidate_text = text_blocks[j][4].strip()
                        if candidate_text:
                            prev_block_text = candidate_text
                            break
                        j -= 1

                    caption_text = prev_block_text.replace("\n", "") if prev_block_text else ""

                    results.append({
                        "Figure_Path": f"File{file_index}_Figure{img_index}",
                        "Caption": caption_text
                    })

                    img_index += 1

    with open(caption_save_path, 'w', encoding='utf-8') as json_file:
        json.dump(results, json_file, indent=4, ensure_ascii=False)

In [ ]:
def chunk_text(text, chunk_size=chunk_size, chunk_overlap=chunk_overlap):
    text_splitter = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    chunks = text_splitter.split_text(text)
    print(f"Total text chunks created: {len(chunks)}")
    return chunks

pdf_chunks = chunk_text(pdf_text)